In [1]:
# basic lib 
import numpy as np
import pandas as pd
import gzip
from collections import defaultdict
import numpy as np

# library for machine learning lib 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
        
        
###  reading the training data
data = []
userRatings = defaultdict(list)

for l in readGz("train.json.gz"):
    data.append(l)

print(len(data))

###  reading the test data

data_test = []

for l in readGz("test_Category.json.gz"):
    data_test.append(l)
print(len(data_test))

df2 = pd.DataFrame(data_test)

200000
14000


In [2]:
#convert data to padas
df = pd.DataFrame.from_dict(data)


In [11]:
### training linearing SVC

X_train = df['reviewText']
y_train = df['categoryID']

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_transformed = tf_transformer.transform(X_train_counts)

linear_svc = svm.LinearSVC()
clf = linear_svc.fit(X_train_transformed,y_train)

calibrated_svc = CalibratedClassifierCV(base_estimator=linear_svc,
                                        cv="prefit")

calibrated_svc.fit(X_train_transformed,y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.001,
     verbose=0),
            cv='prefit', method='sigmoid')

In [12]:
# function to predict test data 

def predict_single(row):
    reviewText = [row['reviewText']] # each document may contain multiple texts, so has to be a list
    p_count = count_vect.transform(reviewText)
    p_tfidf = tf_transformer.transform(p_count)
    pred_this = calibrated_svc.predict_proba(p_tfidf)
    return pred_this


def pred2():
    pred_matrix = []
    
    for idx,row in df2.iterrows():
        
        pred_this = predict_single(row)[0] # prediction is contained in a list
        
        pred_matrix.append(pred_this)
    return pred_matrix


In [13]:
# function call to predict test set

pred_matrix = pred2()

df_pred2 = pd.DataFrame(pred_matrix)

df_pred2['max_label'] = df_pred2.apply(lambda row: (np.argsort(-row[:5])[0]) ,  axis=1 )

prediction = list(df_pred2['max_label'])

In [14]:
#  write the prediction to file 

predictions = open("predictions_Category.txt", 'w')
predictions.write("reviewerID-reviewHash,category\n")
for d,p in zip(data_test,prediction):
    predictions.write(d['reviewerID']+'-'+d['reviewHash']+','+str(p)+'\n')
predictions.close()
